# Charlottesville assessment growth by property value

Where are property values rising fastest? Let's check data from the [Charlottesville Open Data Portal](https://opendata.charlottesville.org/) to see how change in assessment relates to property values. First, we'll calculate proportion change in assessments and bin by percentile of 2020 property value.

Note that we're only including single-family attached and single-family detached properties, since the value of a multi-family property is hard to compare to the value of a single-family home. We also exclude properties built before 2010 to simplify proportion change calculations.

In [1]:
%load_ext google.cloud.bigquery

In [2]:
import altair as alt

In [3]:
%%bigquery assessments
with assessments as (
    select
        ParcelNumb,
        TaxYear,
        TotalValue,
        safe_divide(
            (TotalValue - lag(TotalValue, 1) over (partition by ParcelNumb order by TaxYear)),
            lag(TotalValue, 1) over (partition by ParcelNumb order by TaxYear)
        ) as TotalValuePropChange1y,
        safe_divide(
            (TotalValue - lag(TotalValue, 5) over (partition by ParcelNumb order by TaxYear)),
            lag(TotalValue, 5) over (partition by ParcelNumb order by TaxYear)
        ) as TotalValuePropChange5y,
        safe_divide(
            (TotalValue - lag(TotalValue, 10) over (partition by ParcelNumb order by TaxYear)),
            lag(TotalValue, 10) over (partition by ParcelNumb order by TaxYear)
        ) as TotalValuePropChange10y,
    from `cvilledata.cville_open_data.real_estate_all_assessments`
), baseline as (
    select
        ParcelNumb,
        TotalValue,
        ntile(15) over (order by TotalValue) as TotalValueQuintile,
    from `cvilledata.cville_open_data.real_estate_all_assessments`
    where TaxYear = 2020
)
select distinct
    baseline.TotalValueQuintile,
    percentile_cont(assessments.TotalValuePropChange1y, 0.50) over (partition by baseline.TotalValueQuintile) as TotalValuePropChange1y,
    percentile_cont(assessments.TotalValuePropChange5y, 0.50) over (partition by baseline.TotalValueQuintile) as TotalValuePropChange5y,
    percentile_cont(assessments.TotalValuePropChange10y, 0.50) over (partition by baseline.TotalValueQuintile) as TotalValuePropChange10y,
    min(baseline.TotalValue) over (partition by baseline.TotalValueQuintile) as MinValue,
    max(baseline.TotalValue) over (partition by baseline.TotalValueQuintile) as MaxValue,
    concat(
        "Q",
        baseline.TotalValueQuintile,
        ": $",
        min(baseline.TotalValue / 1000) over (partition by baseline.TotalValueQuintile),
        "k",
        "-",
        max(baseline.TotalValue / 1000) over (partition by baseline.TotalValueQuintile),
        "k"
    ) as QuantileLabel,
from assessments
join baseline using (ParcelNumb)
join `cvilledata.cville_open_data.real_estate_residential_details` residential using (ParcelNumb)
where
    assessments.TaxYear = 2022
    and residential.UseCode in ('Single Family', 'Single Family Attached')
    and residential.YearBuilt < 2010
order by TotalValueQuintile

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:01<00:00, 12.20rows/s]


Now let's look at the results:

In [4]:
assessments

,TotalValueQuintile,TotalValuePropChange1y,TotalValuePropChange5y,TotalValuePropChange10y,MinValue,MaxValue,QuantileLabel
0,1,0.128532,0.154179,0.086839,52700,107700,Q1: $52.7k-107.7k
1,2,0.183295,0.468784,0.510656,110300,162500,Q2: $110.3k-162.5k
2,3,0.116712,0.428834,0.478525,162600,199500,Q3: $162.6k-199.5k
3,4,0.132663,0.413699,0.493580,199600,228200,Q4: $199.6k-228.2k
4,5,0.135221,0.413948,0.482026,228200,256200,Q5: $228.2k-256.2k
5,6,0.131427,0.399409,0.489524,256300,278200,Q6: $256.3k-278.2k
6,7,0.137614,0.405382,0.511241,278200,299200,Q7: $278.2k-299.2k
7,8,0.140625,0.411290,0.505785,299200,324100,Q8: $299.2k-324.1k
8,9,0.116846,0.384615,0.491393,324300,356700,Q9: $324.3k-356.7k
9,10,0.119042,0.400615,0.505261,356900,394800,Q10: $356.9k-394.8k


Let's plot one-year proportion change by property value quantile. This shows that properties in the second quantile increased in value much more than any other quantile:

In [5]:
alt.Chart(assessments).mark_bar().encode(
    x=alt.Y("TotalValuePropChange1y:Q"),
    y=alt.X("QuantileLabel:N", sort=alt.SortField("TotalValueQuintile")),
)

alt.Chart(...)

Does this pattern hold for longer intervals? The distribution of five-year proportion change is somewhat different:

In [6]:
alt.Chart(assessments).mark_bar().encode(
    x=alt.Y("TotalValuePropChange5y:Q"),
    y=alt.X("QuantileLabel:N", sort=alt.SortField("TotalValueQuintile")),
)

alt.Chart(...)

And the pattern is noticeably different for ten-year proportion change, with slightly higher growth in the values of the highest-priced properties.

In [7]:
alt.Chart(assessments).mark_bar().encode(
    x=alt.Y("TotalValuePropChange10y:Q"),
    y=alt.X("QuantileLabel:N", sort=alt.SortField("TotalValueQuintile")),
)

alt.Chart(...)

What does this all tell us?

- One-year assessment changes can be volatile, potentially driven by a small number of sales in a neighborhood. Looking at longer intervals may give more sensible results.
- For longtime property owners, assessments have been growing at a similar pace regardless of property value, excluding the lowest-priced homes.
- New owners of lower-priced homes (but not the lowest-priced homes) may be experiencing some sticker shock in 2022.
- Histograms can change a lot based on binning choices. Use with care.